In [1]:
print("-----------Starting NCBI-----------")

-----------Starting NCBI-----------

In [2]:
from bs4 import BeautifulSoup
import requests

In [3]:
import pandas as pd
import datetime

In [4]:
url="https://ncbiinsights.ncbi.nlm.nih.gov/"

In [5]:
news_dict=[]
response=requests.get(url)
soup=BeautifulSoup(response.content)

for link in soup.find_all("li",{"class":"posted-on"}):
  #print(link)
  if(len(link.text.split(" "))>4):
    news_dict.append({'article link': link.find_all('a')[0].get('href'),'post_date': link.find_all('time')[0].get('datetime')})

In [6]:
newsfeed_df=pd.DataFrame(news_dict)

In [7]:
newsfeed_df=newsfeed_df.reset_index()
newsfeed_df=newsfeed_df.drop(['index'],axis=1)

In [8]:
newsfeed_df.head()

,article link,post_date
0,https://ncbiinsights.ncbi.nlm.nih.gov/2020/09/...,2020-09-04T18:31:13-04:00
1,https://ncbiinsights.ncbi.nlm.nih.gov/2020/09/...,2020-09-01T12:21:41-04:00
2,https://ncbiinsights.ncbi.nlm.nih.gov/2020/08/...,2020-08-25T12:26:41-04:00
3,https://ncbiinsights.ncbi.nlm.nih.gov/2020/08/...,2020-08-21T12:17:25-04:00
4,https://ncbiinsights.ncbi.nlm.nih.gov/2020/08/...,2020-08-17T10:14:23-04:00


In [9]:
newsfeed_df[['post_date','s']]=newsfeed_df['post_date'].str.split('T',expand=True)
newsfeed_df=newsfeed_df.drop(['s'], axis=1)

In [10]:
newsfeed_df['post_date']=pd.to_datetime(newsfeed_df['post_date'])

In [11]:
import datetime as DT
today = DT.date.today()
month_ago = today - DT.timedelta(days=30)
month_ago

Out[59]: datetime.date(2020, 8, 11)

In [12]:
newsfeed_df['past month']='a'
j=0
for i in newsfeed_df['post_date']:
  #temp=datetime.date(i)
  if(i>month_ago):
    newsfeed_df['past month'][j]='yes'
    j=j+1
  else:
    newsfeed_df['past month'][j]='no'
    j=j+1

/local_disk0/tmp/1599769781965-0/PythonShell.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
 import os
/local_disk0/tmp/1599769781965-0/PythonShell.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
 import signal

In [13]:
required_df = newsfeed_df[newsfeed_df['past month'] == 'yes']

In [14]:
article_dict2=[]
article_dict1=[]
for link in required_df['article link']:
  #print(link)
  response=requests.get(link)
  soup=BeautifulSoup(response.content)
  #print(response)

  for link1 in soup.find_all("h1",{"class":"entry-title"}):
    #print("aa")
    article_dict1.append({'article link': link,'headline': link1.text})

for link in required_df['article link']:
  #print(link)
  response=requests.get(link)
  soup=BeautifulSoup(response.content)
  for link2 in soup.find_all("div",{"class":"entry-content"}):
    article_dict2.append({'article link': link,'article':link2.text})

In [15]:
article_df2=pd.DataFrame(article_dict2)
article_df1=pd.DataFrame(article_dict1)

In [16]:
article_df=article_df1.merge(article_df2,how="outer")

In [17]:
article_df.head()

,article link,headline,article
0,https://ncbiinsights.ncbi.nlm.nih.gov/2020/09/...,Hiding sequences in an alignment now available...,\nDo you ever wish there was a quick way to hi...
1,https://ncbiinsights.ncbi.nlm.nih.gov/2020/09/...,NIH Genetic Testing Registry (GTR) now accepti...,\nThe profound impact of the COVID-19 pandemic...
2,https://ncbiinsights.ncbi.nlm.nih.gov/2020/08/...,New Automated Validation in ClinVar Submission,"\nYou, as a submitter, are the beating heart o..."
3,https://ncbiinsights.ncbi.nlm.nih.gov/2020/08/...,Updated and improved collection of RefSeq repr...,\nWe have updated the collection of representa...
4,https://ncbiinsights.ncbi.nlm.nih.gov/2020/08/...,INSDC Statement on SARS-CoV-2 sequence data sh...,\nThe National Library of Medicine and its par...


In [18]:
merge_df=required_df.merge(article_df,how="outer")

In [19]:
merge_df["index"]= merge_df["article"].str.find('medicare advantage')
list_words=['medicare advantage','Medicare Advantage','Medicare advantage']
j=0
for i in list_words:
  if(merge_df["index"][j]==-1):
    merge_df["index"]= merge_df["article"].str.find(i)

In [20]:
merge_df = merge_df[merge_df['index'] != -1]

In [21]:
merge_df=merge_df.drop(['index'], axis=1)

In [22]:
merge_df=merge_df.drop(['past month'], axis=1)

In [23]:
merge_df

,article link,post_date,headline,article


In [24]:
#NCBI website has no articles posted in the past one month which are related to 'medicare advantage' 